# Chapter 7: Convolutional Neural Network
>  
> ### **7.1** 전체 구조  
> CNN도 지금까지 본 신경망과 같이 레고 블록처럼 계층을 조합하여 만들 수 있음  
>  
> convolutional layer와 pooling layer가 새롭게 등장  
>  
> 지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었음  
> &nbsp; &nbsp; &rarr; 이를 fully-connected라고 하며, 이러한 계층을 Affine layer라는 이름으로 구현했음  
>  
> Affine layer를 사용하면 층이 5개인 fully-connected neural network는 다음과 같이 구현할 수 있음  
>  
> ![pic/affine_layer_network](pic/affine_layer_network.png)
>  
> 위와 같이 fully-connected neural network는 Affine layer뒤에 activation function을 갖는 ReLU layer(혹은 Sigmoid layer)가 이어짐  
>  
> 위 그림에서는 Affine-ReLU 조합이 4개 쌓였고, 마지막 5번째 layer는 Affine layer에 이어 softmax layer에서 최종 결과(확률)를 출력  
>  
> ![pic/cnn_example](pic/cnn_example.png)
>  
> 위와 같이 CNN에서는 Conv layer와 Polling layer가 추가됨  
>  
> CNN의 layer는 'Conv-ReLU-(Pooling)'흐름으로 연결됨 (Pooling layer는 생략하기도 함)  
>  
> 지금까지의 'Affine-ReLU'연결이 'Conv-ReLU-(Pooling)'으로 바뀌었다고 생각할 수 있음  
>  
> 위 그림의 CNN에서 주목할 또 다른 점은 출력에 가까운 layer에서는 지금까지의 'Affine-ReLU'구성을 사용할 수 있다는 것  
>  
> 마지막 출력 layer에서는 'Affine-Softmax'조합을 그대로 사용함  
>  
> 이것이 일반적인 CNN에서 흔히 볼 수 있는 구성  

> ### **7.2** Convolutional layer
> CNN에서는 padding, stride 등 CNN 고유의 용어가 등장  
>  
> 각 layer 사이에는 3차원 데이터같이 입체적인 데이터가 흐른다는 점에서 fully-connected network와 다름  
>>  
>> ##### fully-connected layer의 문제점
>> 지금까지 본 fully-connected network에서는 fully-connected layer(Affine layer)를 사용했음  
>>  
>> fully-connected layer에서는 인접하는 layer의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있음  
>>  
>> fully-connected layer의 문제점은 무엇일까?  
>> &nbsp; &nbsp; &rarr; '데이터의 형상이 무시'됨  
>>  
>> 입력 데이터가 이미지인 경우를 예로 들면, 이미지는 통상 세로 &middot; 가로 &middot; 채널로 구성된 3차원 데이터  
>>  
>> fully-connected layer에 입력할 때에는 3차원 데이터를 평평한 1차원 데이터로 평탄화해야 함  
>>  
>> 지금까지의 MNIST 데이터셋을 사용한 사례에서는 shape이 (1, 28, 28)인 이미지(1채널, 세로 28픽셀, 가로 28픽셀)를 1줄로 세운 784개의 데이터를 첫 Affine layer에 입력  
>>  
>> 이미지는 3차원 shape이며, 이 shape에는 중요한 공간적 정보가 담겨 있음  
>> &nbsp; &nbsp; &rarr; 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 관련되어 있거나, 거리가 먼 픽셀끼리는 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것  
>>  
>> 그러나 fully-connected layer는 shape을 무시하고 모든 입력 데이터를 동등한 뉴런(같은 차원의 뉴런)으로 취급하여 shape에 담긴 정보를 살릴 수 없음  
>>  
>> Conv layer는 shape을 유지함  
>>  
>> 이미지도 3차원 데이터로 입력받으며, 다음 계층에도 3차원 데이터로 전달함  
>> &nbsp; &nbsp; &rarr; CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 가능성이 있는 것  
>>  
>> CNN에서는 Conv layer의 입출력 데이터를 feature map이라고도 함  
>>  
>> Conv layer의 입력 데이터를 input feature map, 출력 데이터를 output feature map이라고 함  
>>  
>> 이 책에서는 '입출력 데이터'와 'feature map'을 같은 의미로 사용  

>> ##### 합성곱 연산
>> Conv layer에서는 합성곱 연산을 처리함  
>>  
>> 합성곱 연산은 이미지 처리에서 말하는 필터 연산에 해당  
>>  
>> ![pic/conv_cal_ex](pic/conv_cal_ex.png)
>>  
>> 위 그림과 같이 합성곱 연산은 입력 데이터에 필터를 적용함  
>>  
>> 이 예에서 입력 데이터는 세로 &middot; 가로 방향의 형상을 가졌고, 필터 역시 세로 &middot; 가로 방향의 차원을 가짐  
>>  
>> 데이터와 필터의 shape을 (height, width)로 표기하며, 이 예에서 입력은 (4, 4), 필터는 (3, 3), 출력은 (2, 2)가 됨  
>>  
>> 문헌에 따라 필터를 커널이라 하기도 함  
>>  
>>  
>>  
>> 위 그림의 합성곱 연산  
>> * 합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용  
>>  
>> * 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구함  
>> &nbsp; &nbsp; &rarr; 이 계산을 fused multiply-add, FMA라고 함  
>> * 연산 결과를 출력의 해당 장소에 저장  
>> * 이 과정을 모든 장소에서 수행하면 합성곱 연산의 출력이 완성됨  
>> 
>> ![pic/conv_cal_explain](pic/conv_cal_explain.png)
>>  
>> fully-connected network에는 가중치 매개변수와 편향이 존재, CNN에서는 필터의 매개변수가 그동안의 '가중치'에 해당, 편향까지 포함하면 아래와 같이 표현할 수 있음  
>> 
>> ![pic/cnn_bias](pic/cnn_bias.png)
>>  
>> 위 그림과 같이 편향은 필터를 적용한 후의 데이터에 더해짐  
>>  
>> 편향은 항상 하나($1 \times 1$)만 존재함  
>> 
>> 그 하나의 값을 필터를 적용한 모든 원소에 더함

>> ##### padding
>> padding: 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(0 등)으로 채우는 일, 합성곱 연산에서 자주 이용하는 기법  
>>  
>> 아래 그림은 (4, 4) 크기의 입력 데이터에 폭이 1인 padding을 적용한 모습  
>>  
>> 폭 1짜리 padding은 입력 데이터 사방 1픽셀을 특정 값으로 채우는 것을 말함  
>> 
>> ![pic/padding_ex](pic/padding_ex.png)
>>  
>> 위 그림과 같이 처음에 크기가 (4, 4)인 입력 데이터에 padding이 추가되어 (6, 6)이 됨  
>> 
>> 이 입력에 (3, 3) 크기의 필터를 걸면 (4, 4) 크기의 출력 데이터가 생성  
>>  
>> 이 예에서는 padding을 1로 설정했지만, 2나 3 등 원하는 정수로 설정할 수 있음  
>> 
>> 만약 그림 7-5에 패딩을 2로 설정하면 입력 데이터의 크기는 (8, 8)이 되고, 3으로 설정하면 (10, 10)이 됨  
>>  
>> padding은 주로 출력 크기를 조정할 목적으로 사용함  
>> 
>> 예를 들어 (4, 4) 입력 데이터에 (3, 3) 필터를 적용하면 출력은 (2, 2)가 되어 입력보다 2만큼 줄어듦  
>> &nbsp; &nbsp; &rarr; 이는 합성곱 연산을 몇 번이나 되풀이하는 심층 신경망에서는 문제가 될 수 있음  
>> &nbsp; &nbsp; &rarr; 합성곱 연산을 거칠 때마다 크기가 작아지면 어느 시점에서는 출력 크기가 1이 되어버림  
>> &nbsp; &nbsp; &rarr; 더 이상은 합성곱 연산을 적용할 수 없음  
>>  
>> 이러한 사태를 막기 위해 padding을 사용  
>> &nbsp; &nbsp; &rarr; 입력 데이터의 공간적 크기를 고정한 채로 다음 layer에 전달할 수 있음  

>> ##### stride
>> stride: 필터를 적용하는 위치의 간격  
>> 
>> 지금까지 본 예는 모두 stride가 1이었지만, 예를 들어 stride를 2로 하면 필터를 적용하는 윈도우가 두 칸씩 이동함  
>> 
>> ![pic/stride_2_ex](pic/stride_2_ex.png)
>> 
>> 위 그림에서는 크기가 (7, 7)인 입력 데이터에 stride를 2로 설정한 필터를 적용함  
>> 
>> 이처럼 stride는 필터를 적용하는 간격을 지정함  
>> 
>> stride를 2로 하니 출력은 (3, 3)이 됨  
>> &nbsp; &nbsp; &rarr; 이처럼 stride를 키우면 출력 크기는 작아짐
>> 
>> 한편, padding을 크게 하면 출력 크기가 커짐  
>> 
>> 이러한 관계를 수식화하면 다음과 같음
>> 
>> * $ OH = \frac{H+2P-FH}{S} + 1 $
>> 
>> * $ OW = \frac{W+wP-FW}{S} + 1 $ &nbsp; &nbsp; &nbsp; [식 7.1]
>>      * H, W: 입력 크기 (height, width)
>>      * FH, FW: 필터 크기 (Filter Height, Filter Width)
>>      * OH, OW: 출력 크기 (Output Height, Output Width)
>>      * P: padding
>>      * S: stride
>>  
>> 연습
>> * 예1: 그림 7-6  
>> &nbsp; &nbsp; &nbsp; &nbsp; 입력: (4,4), padding: 1, stride: 1, 필터: (3,3)  
>> $$
>> OH = \frac{4+2 \cdot 1-3}{1} + 1 = 4
>> $$
>> $$
>> OW = \frac{4+2 \cdot 1-3}{1} + 1 = 4
>> $$
>>  
>> * 예2: 그림 7-7  
>> &nbsp; &nbsp; &nbsp; &nbsp; 입력: (7,7), padding: 0, stride: 2, 필터: (3,3)  
>> $$
>> OH = \frac{7+2 \cdot 0-3}{2} + 1 = 3
>> $$
>> $$
>> OW = \frac{7+2 \cdot 0-3}{2} + 1 = 3
>> $$
>> 
>> * 예3  
>> &nbsp; &nbsp; &nbsp; &nbsp; 입력: (28,31), padding: 2, stride: 3, 필터: (5,5)  
>> $$
>> OH = \frac{28+2 \cdot 2-5}{3} + 1 = 10
>> $$
>> $$
>> OW = \frac{31+2 \cdot 2-5}{3} + 1 = 11
>> $$
>>  
>> 위의 예에서처럼 식 7.1에 단순히 값을 대입하기만 하면 출력 크기를 구할 수 있음  
>> 
>> 단 식 7.1의 $ \frac{H+2P-FH}{S} $와 $ \frac{W+wP-FW}{S} $가 정수로 나누어 떨어지는 값이어야 한다는 점에 주의  
>> 
>> 출력 크기가 정수가 아니면 오류를 내는 등의 대응을 해주어야 함  
>> 
>> 딥러닝 프레임워크 중에는 값이 나누어 떨어지지 않을 때는 가장 가까운 정수로 반올림하는 등, 특별히 에러를 내지 않고 진행하도록 구현하는 경우도 있음  

>> ##### 3차원 데이터의 합성곱 연산
>> 지금까지 2차원 shape을 다루는 합성곱 연산을 살펴봄  
>> 
>> 그러나 이미지만 해도 height &middot; width에 더해 채널까지 고려한 3차원 데이터임  
>> 
>> 지금까지 살펴본 순서대로 채널까지 고려한 3차원 데이터를 다루는 합성곱 연산을 살펴볼 예정  
>> 
>> ![pic/3dim_conv_ex](pic/3dim_conv_ex.png)
>> 
>> ![pic/order_3dim_conv](pic/order_3dim_conv.png)
>> 
>> 위 그림은 3차원 데이터의 합성곱 연산 예  
>> 
>> 2차원일 때(그림 7-3)와 비교하면, 길이 방향(채널 방향)으로 feature map이 늘어남  
>> 
>> 채널 방향으로 feature map이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻음
>>  
>> 3차원 합성곱 연산에서 주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것  
>> 
>> 필터 자체의 크기는 원하는 값으로 설정할 수 있음 (단, 모든 채널의 필터가 같은 크기여야 함)  

>> ##### 블록으로 생각하기
>> 3차원 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉬움  
>> 
>> 블록은 3차원 직육면체임  
>> 
>> 3차원 데이터를 다차원 배열로 나타낼 때는 (channel, height, width)로 표현  
>> 
>> ![pic/conv_block](pic/conv_block.png)
>> 
>> 위 그림에서 출력 데이터는 한 장의 feature map  
>> &nbsp; &nbsp; &rarr; 한 장의 feature map = 채널이 1개인 feature map  
>> 
>> 합성곱 연산의 출력으로 다수의 채널을 내보내려면 어떻게 해야 할까?  
>> &nbsp; &nbsp; &rarr; 필터(가중치)를 여러 개 사용  
>> 
>> 그림으로 나타내면 아래와 같음  
>>  
>> ![pic/conv_many_filter](pic/conv_many_filter.png)
>> 
>> 위 그림과 같이 필터를 FN개 적용하면 출력 feature map도 FN개가 생성  
>> 
>> 그리고 그 FN개의 맵을 모으면 shape이 (FN, OH, OW)인 블록이 완성  
>>  
>> 이 완성된 블록을 다음 계층으로 넘기는 것이 CNN의 처리 흐름  
>>  
>> 살펴본 것처럼 합성곱 연산에서는 필터의 수도 고려해야 함  
>> 
>> 그렇기 때문에 필터의 가중치 데이터는 4차원 데이터이며 (output channel, input channel, height, width)로 나타냄  
>> &nbsp; &nbsp; &rarr; ex) 채널 수 $3$, 크기 $5 \times 5$인 필터가 $20$개 있다면 (20, 3, 5, 5)로 나타냄  
>> 
>> 합성곱 연산에도 편향이 쓰임  
>> 
>> 이를 그림으로 나타내면, 
>> 
>> ![pic/conv_cal_flow](pic/conv_cal_flow.png)
>> 
>> 위 그림에서 보듯 편향은 채널 하나에 값 하나씩으로 구성됨  
>> 
>> 이 예에서는 편향의 shape은 (FN, 1, 1)이고, 필터의 출력 결과의 형사은 (FN, OH, OW)임  
>> 
>> 이 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN, OH, OW) 블록의 대응 채널의 원소 모두에 더해짐  

>> ##### 배치 처리
>> 신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했음  
>> 
>> fully-connected network를 구현하면서는 이 방식을 지원하여 처리 효율을 높이고, 미니배치 방식의 학습도 지원하도록 했음  
>> 
>> 합성곱 연산도 마찬가지로 배치 처리를 지원  
>> &nbsp; &nbsp; &rarr; 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장함  
>> 
>> 구체적으로는 데이터를 (number of data, channel, height, width)로 저장  
>> 
>> 데이터가 N개일 때 배치 처리한다면 데이터 형태가 다음과 같이 구성  
>> 
>> ![pic/conv_cal_batch](pic/conv_cal_batch.png)
>> 
>> 배치 처리 시의 데이터 흐름을 나타낸 위 그림을 보면 각 데이터의 선두에 배치용 차원을 추가함  
>> 
>> 이처럼 데이터는 4차원 shape을 가진 채 각 layer를 타고 흐름  
>> 
>> 여기서 주의할 점으로는 신경망에 4차원 데이터가 하나 흐르 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것  
>> 
>> 즉, N회 분의 처리를 한 번에 수행하는 것

> ### **7.3** Pooling layer
>> 
>> pooling: hegiht &middot; width 방향의 공간을 줄이는 연산  
>> 
>> ![pic/max_pooling_flow](pic/max_pooling_flow.png)
>> 
>> 위 그림과 같이 $2 \times 2$영역을 원소 하나로 집약하여 공간 크기를 줄임  
>>  
>> 위 그림은 $ 2 \times 2$ max pooling을 stride 2로 처리하는 순서
>> 
>> max pooling: 최댓값을 구하는 연산  
>> 
>> '$2 \times 2$'는 대상 영역의 크기를 뜻함  
>> 
>> 즉 $2 \times 2$ max pooling은 위 그림과 같이 $2 \times 2$ 크기의 영역에서 가장 큰 원소 하나를 꺼냄  
>> 
>> stride는 이 예에서는 2로 설정했으므로 $2 \times 2$ 윈도우가 원소 2칸 간격으로 이동  
>> 
>> 참고로, pooling의 윈도우 크기와 stride는 같은 값으로 설정하는 것이 보통  
>> &nbsp; &nbsp; &rarr; ex) 윈도우가 $3 \times 3$ 이면 stride는 3으로, 윈도우가 $4 \times 4$이면 stride를 4 설정  
>>  
>> pooling은 max pooling 외에도 average pooling 등이 있음  
>> 
>> max pooling은 대상 영역에서 최댓값을 취하는 연산인 반면, average pooling은 대상 영역의 평균을 계산함  
>> 
>> 이미지 인식 분야에서는 주로 max pooling을 사용함  
>> &nbsp; &nbsp; &rarr; 이 책에서 pooling layer라고 하면 max pooling을 말하는 것  
>> 
>> ##### pooling layer의 특징
>> * 학습해야 할 매개변수가 없음  
>> &nbsp; &nbsp; pooling layer는 Conv layer와 달리 학습해야 할 매개변수가 없음  
>> &nbsp; &nbsp; pooling은 대상 영역에서 최댓값이나 평균을 취하는 명확한 처리이므로 특별히 학습할 것이 없음  
>> 
>> * 채널 수가 변하지 않음  
>> &nbsp; &nbsp; pooling 연산은 입력 데이터의 채널 수 그대로 출력 데이터로 내보냄  
>> &nbsp; &nbsp; pooling 연산은 아래 그림처럼 채널마다 독립적으로 계산하기 때문  
>>
>> ![pic/pooling_feature](pic/pooling_feature.png)
>> 
>> * 입력의 변화에 영향을 적게 받음 (강건함)  
>> &nbsp; &nbsp; 입력 데이터가 조금 변해도 pooling의 결과는 잘 변하지 않음  
>> &nbsp; &nbsp; 예를 들어 다음 그림은 입력 데이터의 차이(데이터가 오른쪽으로 1칸씩 이동)를 pooling이 흡수해 사라지게 하는 모습을 보여줌  
>> 
>> ![pic/pooling_feature_2](pic/pooling_feature_2.png)
